In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

# Load the train and test dataset
train_generator = train_datagen.flow_from_directory(
    'dataset/jajanan/train',
    batch_size=32,
    target_size=(224, 224),
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/jajanan/test',
    batch_size=32,
    target_size=(224, 224),
    class_mode='categorical'
)

Found 1884 images belonging to 8 classes.
Found 473 images belonging to 8 classes.


In [7]:
from keras.callbacks import LearningRateScheduler

# Define a learning rate schedule function
def lr_schedule(epoch):
    if epoch < 5:
        return 0.0005
    else:
        return 0.0001

# Create a LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [8]:
#Transfer Learning InceptionV3
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
import numpy as np

# Load InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

last_layer = base_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# Add custom layers
x = layers.Flatten()(last_output)
x = layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(8, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=int(np.ceil(train_generator.n / float(32))),
    epochs=15,
    validation_data=test_generator,
    validation_steps=int(np.ceil(test_generator.n / float(32))),
    verbose=2,
    callbacks=[lr_scheduler]
)

last layer output shape:  (None, 12, 12, 768)
Epoch 1/15


In [ ]:
import datetime
model.save('model_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))